
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1FhYi4YTFXYHt8Ea8Tz-RjJWrjWZNvfRo?usp=sharing)

In [1]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [2]:
!pip install 'temporal_fusion_transformer[training] @ git+https://github.com/aaarrti/temporal_fusion_transformer.git@develop'
!pip install "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

  Cloning https://github.com/aaarrti/temporal_fusion_transformer.git (to revision develop) to /tmp/pip-install-7z8deuie/temporal-fusion-transformer_a1b9f8fe47a34571ac203feb761c74f5
  Running command git clone --filter=blob:none --quiet https://github.com/aaarrti/temporal_fusion_transformer.git /tmp/pip-install-7z8deuie/temporal-fusion-transformer_a1b9f8fe47a34571ac203feb761c74f5
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/aaarrti/temporal_fusion_transformer.git to commit 30ae1f921d81846a19e9d18633a5ad46b3b0b867
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for temporal_fusion_transformer: filename=temporal_fusion_transformer-0.0.1.dev1-py3-none-any.whl size=28705 sha256=72b7c1833b8e2f0505bef9e61ef72380b700013836bdae92e1850d

First, we'd need to upload dataset to GDrive

In [1]:
# TODO: use GCS bucket
from google.colab import drive

drive.mount("/content/drive")

data_path = "/content/drive/MyDrive/datasets"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import jax
from absl_extra import logging_utils
import temporal_fusion_transformer as tft

tf.config.set_visible_devices([], "GPU")
jax.config.update("jax_debug_nans", True)
logging_utils.setup_logging(log_level="INFO")
jax.devices()

[gpu(id=0)]

In [4]:
config = tft.get_config("electricity")
config

fixed_params:
  input_known_categorical_idx:
  - 3
  - 4
  - 5
  - 6
  input_known_real_idx:
  - 1
  - 2
  input_observed_idx: []
  input_static_idx:
  - 0
  known_categories_sizes:
  - 31
  - 24
  - 7
  - 12
  num_encoder_steps: 168
  num_outputs: 1
  static_categories_sizes:
  - 369
  total_time_steps: 192
hyperparams:
  dropout_rate: 0.1
  latent_dim: 160
  num_attention_heads: 4
  num_decoder_blocks: 1
  quantiles:
  - 0.1
  - 0.5
  - 0.9
optimizer:
  clipnorm: 0.1
  decay_alpha: 0.05
  decay_steps: 0.8
  ema: 0.99
  learning_rate: 0.001
prng_seed: 69
shuffle_buffer_size: 1024

In [7]:
tft.training.train_on_single_device(
    data_dir=data_path,
    batch_size=64,
    experiment_name="electricity",
    mixed_precision=True,
    config=config,
    save_path="/content/drive/MyDrive/models",
    jit_module=True,
)

FloatingPointError: ignored